Read data

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("E:/study_e/交通事故安全/code/mgtwr/")

In [2]:
data = pd.read_csv(r'E:\study_e\交通事故安全\code\mgtwr\data\example.csv')
data.head(2)

,longitude,latitude,t,x1,x2,y
0,0,0,0,0.771321,0.895098,10.656550
1,1,0,0,0.020752,0.633729,5.692754


In [3]:
coords = data[['longitude', 'latitude']]
t = data[['t']]
X = data[['x1', 'x2']]
y = data[['y']]

GWR model

In [11]:
from mgtwr_local.sel import SearchGWRParameter
from mgtwr_local.model import GWR

In [12]:
sel = SearchGWRParameter(coords, X, y, kernel='gaussian', fixed=True)
bw = sel.search(bw_max=40, verbose=True, time_cost=True)

bw: 15.0 , score: 18778.49
bw: 10.0 , score: 18764.75
bw: 6.0 , score: 18699.21
bw: 4.0 , score: 18506.22
bw: 2.0 , score: 17786.86
bw: 2.0 , score: 17786.86
time cost: 0:00:3.111


In [13]:
gwr = GWR(coords, X, y, bw, kernel='gaussian', fixed=True).fit()
print(gwr.R2)

0.5935790327518


In [14]:
def summaryGWR(Results):
    XNames = ["X"+str(i) for i in range(Results.k)]
    
    summary = "%s\n" %('Geographically weighted regression (GWR) Results')
    summary += '-' * 75 + '\n'

    if Results.fixed:
        summary += "%-50s %20s\n" % ('Spatial kernel:', 'Fixed ' + Results.kernel)
    else:
        summary += "%-54s %20s\n" % ('Spatial kernel:', 'Adaptive ' + Results.kernel)
    
    summary += "%-62s %12.3f\n" % ('Temporal Bandwidth used:',  Results.bw)

    summary += "\n%s\n" % ('Diagnostic information')
    summary += '-' * 75 + '\n'
    
    if Results.kernel == 'gaussian':
        
        summary += "%-62s %12.3f\n" % ('Residual sum of squares:', Results.RSS)
        summary += "%-62s %12.3f\n" % ('Effective number of parameters (trace(S)):', Results.tr_S)
        summary += "%-62s %12.3f\n" % ('Degree of freedom (n - trace(S)):', Results.df_model)
        summary += "%-62s %12.3f\n" % ('Sigma estimate:', np.sqrt(Results.sigma2))
        summary += "%-62s %12.3f\n" % ('Log-likelihood:', Results.llf)
        summary += "%-62s %12.3f\n" % ('AIC:', Results.aic)
        summary += "%-62s %12.3f\n" % ('AICc:', Results.aicc)
        summary += "%-62s %12.3f\n" % ('BIC:', Results.bic)
        summary += "%-62s %12.3f\n" % ('R2:', Results.R2)
    else:
        summary += "%-62s %12.3f\n" % ('Effective number of parameters (trace(S)):', Results.tr_S)
        summary += "%-62s %12.3f\n" % ('Degree of freedom (n - trace(S)):', Results.df_model)
        summary += "%-62s %12.3f\n" % ('Log-likelihood:', Results.llf)
        summary += "%-62s %12.3f\n" % ('AIC:', Results.aic)
        summary += "%-62s %12.3f\n" % ('AICc:', Results.aicc)
        summary += "%-62s %12.3f\n" % ('BIC:', Results.bic)
        #summary += "%-60s %12.6f\n" % ('Percent deviance explained:', 0)


    summary += "%-62s %12.3f\n" % ('Adj. alpha (95%):', Results.adj_alpha[1])
    summary += "%-62s %12.3f\n" % ('Adj. critical t value (95%):', Results.critical_tval(Results.adj_alpha[1]))

    summary += "\n%s\n" % ('Summary Statistics For GWR Parameter Estimates')
    summary += '-' * 75 + '\n'
    summary += "%-20s %10s %10s %10s %10s %10s\n" % ('Variable', 'Mean' ,'STD', 'Min' ,'Median', 'Max')
    summary += "%-20s %10s %10s %10s %10s %10s\n" % ('-'*20, '-'*10 ,'-'*10, '-'*10 ,'-'*10, '-'*10)
    for i in range(Results.k):
        summary += "%-20s %10.3f %10.3f %10.3f %10.3f %10.3f\n" % (XNames[i], np.mean(Results.betas[:,i]) ,np.std(Results.betas[:,i]),np.min(Results.betas[:,i]) ,np.median(Results.betas[:,i]), np.max(Results.betas[:,i]))

    summary += '=' * 75 + '\n'

    return summary

In [17]:
print(summaryGWR(gwr))

Geographically weighted regression (GWR) Results
---------------------------------------------------------------------------
Spatial kernel:                                          Fixed gaussian
Temporal Bandwidth used:                                              2.000

Diagnostic information
---------------------------------------------------------------------------
Residual sum of squares:                                        2901501.328
Effective number of parameters (trace(S)):                           24.069
Degree of freedom (n - trace(S)):                                  1703.931
Sigma estimate:                                                      41.265
Log-likelihood:                                                   -8868.006
AIC:                                                              17786.151
AICc:                                                             17786.859
BIC:                                                              17765.831
R2:                

MGWR model

In [10]:
from mgtwr_local.sel import SearchMGWRParameter
from mgtwr_local.model import MGWR

In [11]:
sel_multi = SearchMGWRParameter(coords, X, y, kernel='gaussian', fixed=True)
bws = sel_multi.search(multi_bw_max=[1000], verbose=True, time_cost=True, tol_multi=3.0e-5)

Current iteration: 1 ,SOC: 0.0033171
Bandwidths: 986.8, 965.5, 0.7
Current iteration: 2 ,SOC: 5.64e-05
Bandwidths: 986.8, 986.8, 0.7
Current iteration: 3 ,SOC: 4.27e-05
Bandwidths: 986.8, 986.8, 0.7
Current iteration: 4 ,SOC: 3.22e-05
Bandwidths: 986.8, 986.8, 0.7
Current iteration: 5 ,SOC: 2.43e-05
Bandwidths: 986.8, 986.8, 0.7
time cost: 0:01:20.431


In [12]:
mgwr = MGWR(coords, X, y, sel_multi, kernel='gaussian', fixed=True).fit()
print(mgwr.R2)

0.7045779853867871


If you already know bws, you can also do the following

In [13]:
class sel_multi:
    def __init__(self, bws):
        self.bws = bws

        
selector = sel_multi(bws)
mgwr = MGWR(coords, X, y, selector, kernel='gaussian', fixed=True).fit()
print(mgwr.R2)

0.7045779853867871


GTWR model

In [4]:
from mgtwr_local.sel import SearchGTWRParameter
from mgtwr_local.model import GTWR

In [5]:
sel = SearchGTWRParameter(coords, t, X, y, kernel='gaussian', fixed=True)
bw, tau = sel.search(tau_max=20, verbose=True, time_cost=True)

bw:  5.9 , tau:  19.9 , score:  18095.04059255282
bw:  3.7 , tau:  19.9 , score:  17608.38596885707
bw:  2.3 , tau:  10.1 , score:  16461.58709937909
bw:  1.4 , tau:  3.8 , score:  14817.811620052908
bw:  0.9 , tau:  1.4 , score:  13780.792562049754
bw:  0.9 , tau:  1.4 , score:  13780.792562049754
bw:  0.9 , tau:  1.4 , score:  13780.792562049754
bw:  0.9 , tau:  1.4 , score:  13780.792562049754
bw:  0.9 , tau:  1.4 , score:  13780.792562049754
time cost: 0:01:40.489


In [6]:
gtwr = GTWR(coords, t, X, y, bw, tau, kernel='gaussian', fixed=True).fit()
print(gtwr.R2)

0.9829884630503501


In [23]:
X

,x1,x2
0,0.771321,0.895098
1,0.020752,0.633729
2,0.633648,0.462768
3,0.748804,0.090788
4,0.498507,0.982153
...,...,...
1723,0.289875,0.741360
1724,0.921317,0.962916
1725,0.222337,0.055764
1726,0.633053,0.825171


In [21]:
gtwr.X

array([[1.        , 0.77132064, 0.89509824],
       [1.        , 0.02075195, 0.63372887],
       [1.        , 0.63364823, 0.46276849],
       ...,
       [1.        , 0.22233723, 0.05576403],
       [1.        , 0.63305281, 0.82517067],
       [1.        , 0.05729903, 0.87299008]])

In [22]:
gtwr.X.shape

(1728, 3)

In [18]:
gtwr.betas

array([[ 4.61469855,  5.45243831,  2.99766617],
       [ 2.32808632,  4.72600524,  8.75860259],
       [-0.52334178,  5.58047125, 16.77459138],
       ...,
       [ 8.90916936, -7.28382225, 50.68065614],
       [ 8.44766537, 10.24180477, 24.51544434],
       [10.99289026, 12.95482629, 12.81980239]])

In [19]:
gtwr.betas.shape

(1728, 3)

In [30]:
columns_list = list(X.columns)
columns_list.insert(0,'const')
columns_list

['const', 'x1', 'x2']

In [31]:
pd.DataFrame(gtwr.betas , columns= columns_list)

,const,x1,x2
0,4.614699,5.452438,2.997666
1,2.328086,4.726005,8.758603
2,-0.523342,5.580471,16.774591
3,3.704357,2.380499,12.812832
4,6.740300,1.208885,6.016846
...,...,...,...
1723,7.839976,-3.918450,59.958556
1724,10.822673,-17.987315,62.384594
1725,8.909169,-7.283822,50.680656
1726,8.447665,10.241805,24.515444


In [8]:
def summaryGTWR(Results):
    XNames = ["X"+str(i) for i in range(Results.k)]
    
    summary = "%s\n" %('Spatiotemporal Weighted Regression (GTWR) Results')
    summary += '-' * 75 + '\n'

    if Results.fixed:
        summary += "%-50s %20s\n" % ('Spatial kernel:', 'Fixed ' + Results.kernel)
    else:
        summary += "%-54s %20s\n" % ('Spatial kernel:', 'Adaptive ' + Results.kernel)

    summary += "%-62s %12.3f\n" % ('Model tau used:', Results.tau)
        
    summary += "%-62s %12.3f\n" % ('Spatial Bandwidth used:', Results.bw_s)
    
    summary += "%-62s %12.3f\n" % ('Temporal Bandwidth used:',  Results.bw_t)

    summary += "\n%s\n" % ('Diagnostic information')
    summary += '-' * 75 + '\n'
    
    if Results.kernel == 'gaussian':
        
        summary += "%-62s %12.3f\n" % ('Residual sum of squares:', Results.RSS)
        summary += "%-62s %12.3f\n" % ('Effective number of parameters (trace(S)):', Results.tr_S)
        summary += "%-62s %12.3f\n" % ('Degree of freedom (n - trace(S)):', Results.df_model)
        summary += "%-62s %12.3f\n" % ('Sigma estimate:', np.sqrt(Results.sigma2))
        summary += "%-62s %12.3f\n" % ('Log-likelihood:', Results.llf)
        summary += "%-62s %12.3f\n" % ('AIC:', Results.aic)
        summary += "%-62s %12.3f\n" % ('AICc:', Results.aicc)
        summary += "%-62s %12.3f\n" % ('BIC:', Results.bic)
        summary += "%-62s %12.3f\n" % ('R2:', Results.R2)
    else:
        summary += "%-62s %12.3f\n" % ('Effective number of parameters (trace(S)):', Results.tr_S)
        summary += "%-62s %12.3f\n" % ('Degree of freedom (n - trace(S)):', Results.df_model)
        summary += "%-62s %12.3f\n" % ('Log-likelihood:', Results.llf)
        summary += "%-62s %12.3f\n" % ('AIC:', Results.aic)
        summary += "%-62s %12.3f\n" % ('AICc:', Results.aicc)
        summary += "%-62s %12.3f\n" % ('BIC:', Results.bic)
        #summary += "%-60s %12.6f\n" % ('Percent deviance explained:', 0)


    summary += "%-62s %12.3f\n" % ('Adj. alpha (95%):', Results.adj_alpha[1])
    summary += "%-62s %12.3f\n" % ('Adj. critical t value (95%):', Results.critical_tval(Results.adj_alpha[1]))

    summary += "\n%s\n" % ('Summary Statistics For GTWR Parameter Estimates')
    summary += '-' * 75 + '\n'
    summary += "%-20s %10s %10s %10s %10s %10s\n" % ('Variable', 'Mean' ,'STD', 'Min' ,'Median', 'Max')
    summary += "%-20s %10s %10s %10s %10s %10s\n" % ('-'*20, '-'*10 ,'-'*10, '-'*10 ,'-'*10, '-'*10)
    for i in range(Results.k):
        summary += "%-20s %10.3f %10.3f %10.3f %10.3f %10.3f\n" % (XNames[i], np.mean(Results.betas[:,i]) ,np.std(Results.betas[:,i]),np.min(Results.betas[:,i]) ,np.median(Results.betas[:,i]), np.max(Results.betas[:,i]))

    summary += '=' * 75 + '\n'

    return summary

In [10]:
print(summaryGTWR(gtwr) )

Spatiotemporal Weighted Regression (STWR) Results
---------------------------------------------------------------------------
Spatial kernel:                                          Fixed gaussian
Model tau used:                                                       1.400
Spatial Bandwidth used:                                               0.900
Temporal Bandwidth used:                                              0.761

Diagnostic information
---------------------------------------------------------------------------
Residual sum of squares:                                         121447.959
Effective number of parameters (trace(S)):                          529.125
Degree of freedom (n - trace(S)):                                  1198.875
Sigma estimate:                                                      10.065
Log-likelihood:                                                   -6126.104
AIC:                                                              13312.459
AICc:             

MGTWR model

In [17]:
from mgtwr_local.sel import SearchMGTWRParameter
from mgtwr_local.model import MGTWR

In [18]:
sel_multi = SearchMGTWRParameter(coords, t, X, y, kernel='gaussian', fixed=True)
bws = sel_multi.search(multi_bw_min=[0.1], verbose=True, tol_multi=1.0e-4, time_cost=True)

Current iteration: 1 ,SOC: 0.0025274
Bandwidths: 0.7, 0.7, 0.5
taus: 1.3,0.8,0.8
Current iteration: 2 ,SOC: 0.0011033
Bandwidths: 0.9, 0.7, 0.5
taus: 3.0,0.4,0.8
Current iteration: 3 ,SOC: 0.0005365
Bandwidths: 0.9, 0.7, 0.5
taus: 3.4,0.2,0.8
Current iteration: 4 ,SOC: 0.0003
Bandwidths: 0.9, 0.7, 0.5
taus: 3.4,0.2,0.8
Current iteration: 5 ,SOC: 0.0001986
Bandwidths: 0.9, 0.7, 0.5
taus: 3.6,0.2,0.8
Current iteration: 6 ,SOC: 0.0001415
Bandwidths: 0.9, 0.7, 0.5
taus: 3.6,0.2,0.8
Current iteration: 7 ,SOC: 0.0001052
Bandwidths: 0.9, 0.7, 0.5
taus: 3.6,0.2,0.8
Current iteration: 8 ,SOC: 7.99e-05
Bandwidths: 0.9, 0.7, 0.5
taus: 3.6,0.2,0.8
time cost: 0:19:26.941


In [19]:
mgtwr = MGTWR(coords, t, X, y, sel_multi, kernel='gaussian', fixed=True).fit()
print(mgtwr.R2)

0.9972924820674222


If you already know bws, you can also do the following

In [20]:
class sel_multi:
    def __init__(self, bws):
        self.bws = bws

        
selector = sel_multi(bws)
mgtwr = MGTWR(coords, t, X, y, selector, kernel='gaussian', fixed=True).fit()
print(mgtwr.R2)

0.9972924820674222
